In [2]:
%cd e:\kaggle\wsdm
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from importlib import reload
import gc
import sys

e:\kaggle\wsdm


In [7]:
%cd d:\jupyter-workdir\notebook
from mylibs.WSDN_FUNC import *
import mylibs.WSDN_FUNC as func
%cd -

d:\jupyter-workdir\notebook
e:\kaggle\wsdm


In [17]:
reload(func)

<module 'mylibs.WSDN_FUNC' from 'd:\\jupyter-workdir\\notebook\\mylibs\\WSDN_FUNC.py'>

In [ ]:
test_data = pd.DataFrame({"msno":[1,2,3,4,5,1,2,3],"num1":[4,5,6,7,8,1,2,3],"num2":[5,6,6,7,8,1,4,3], "date":["2017-1-1","2017-2-1","2017-3-1","2017-4-1","2017-5-1","2017-6-1","2017-7-1","2017-8-1"]})
test_data["date"] = pd.to_datetime(test_data["date"], format='%Y-%m-%d')
num_cols = [col for col in test_data.columns if col != "msno"]
group_data = test_data.groupby("msno")
sum_data=test_data.groupby("msno")[num_cols].sum().reset_index()
mean_data=test_data.groupby("msno")[num_cols].mean().reset_index(drop=True)
concat_data=pd.concat([sum_data, mean_data], axis=1)
concat_data

In [ ]:
test_data.sort_values(by=['msno'], ascending=[True]).reset_index(drop=True).set_index("msno").truncate(before="3")

In [ ]:
def imin(data):
    cumsum = 0
    for i in range(len(data) - 1):
        x = data.iloc[i]
        y = data.iloc[i+1]
        cumsum += 1 if x <= y else -1
    return cumsum
group_data[["num1", "num2"]].agg(imin).reset_index()

In [ ]:
test_data.set_index("date").truncate(before="2017-03-01").reset_index().groupby("msno")[num_cols].sum().reset_index()

In [ ]:
test_data.msno.value_counts().reset_index().rename(columns={"index":"msno", "msno":"log_count"}).groupby("msno")["log_count"].sum().reset_index()

In [ ]:
# user_log num count after 3-1
df_iter = pd.read_csv('user_logs.csv/user_logs.csv', low_memory=False, iterator=True, chunksize=10000000)
last_user_logs_count = []
i = 0 
for df in df_iter:
    if len(df)>0:
        print(df.shape)
        p = Pool(cpu_count())
        df = p.map(func.user_log_num_count, np.array_split(df, cpu_count()))   
        df = pd.concat(df, axis=0, ignore_index=True).reset_index(drop=True)
        df = func.user_log_num_count2(df)
        p.close(); p.join()
        last_user_logs_count.append(df)
        print('...', df.shape)
        df = []
        gc.collect()

last_user_logs_count = pd.concat(last_user_logs_count, axis=0, ignore_index=True).reset_index(drop=True)
last_user_logs_count = func.user_log_num_count2(last_user_logs_count)

(10000000, 9)
... (661542, 2)
(10000000, 9)
... (585393, 2)
(10000000, 9)
... (649722, 2)
(10000000, 9)
... (662229, 2)
(10000000, 9)
... (613887, 2)
(10000000, 9)
... (580158, 2)
(10000000, 9)
... (634413, 2)
(10000000, 9)
... (662149, 2)
(10000000, 9)
... (563477, 2)
(10000000, 9)
... (595551, 2)
(10000000, 9)
... (662440, 2)
(10000000, 9)
... (661494, 2)
(10000000, 9)
... (546180, 2)
(10000000, 9)
... (596466, 2)
(10000000, 9)
... (663025, 2)
(10000000, 9)
... (631776, 2)
(10000000, 9)
... (596415, 2)
(10000000, 9)
... (640445, 2)
(10000000, 9)
... (662538, 2)
(10000000, 9)
... (600753, 2)
(10000000, 9)
... (593493, 2)
(10000000, 9)
... (661585, 2)
(10000000, 9)
... (628885, 2)
(10000000, 9)
... (578837, 2)
(10000000, 9)
... (626743, 2)
(10000000, 9)
... (662359, 2)
(10000000, 9)
... (624582, 2)
(10000000, 9)
... (579140, 2)
(10000000, 9)
... (648529, 2)
(10000000, 9)
... (661305, 2)
(10000000, 9)
... (607222, 2)
(10000000, 9)
... (633829, 2)
(10000000, 9)
... (661516, 2)
(10000000,

In [20]:
last_user_logs_count.count()

msno            1103894
log_count_1m    1103894
dtype: int64

In [21]:
#last_user_logs_count.to_csv('extra_features3.csv', index=False) # after 2-1
last_user_logs_count.to_csv('extra_features5.csv', index=False) # after 3-1

In [ ]:
# user_logs trend
df_iter = pd.read_csv('user_logs.csv/user_logs.csv', low_memory=False, iterator=True, chunksize=10000000)
last_user_logs_trend = []
i = 0 
for df in df_iter:
    if len(df)>0:
        print(df.shape)
        p = Pool(cpu_count())
        df = p.map(user_log_trend, np.array_split(df, cpu_count()))   
        df = pd.concat(df, axis=0, ignore_index=True).reset_index(drop=True)
        df = user_log_trend2(df)
        p.close(); p.join()
        last_user_logs_trend.append(df)
        print('...', df.shape)
        df = []
        gc.collect()

last_user_logs_trend = pd.concat(last_user_logs_trend, axis=0, ignore_index=True).reset_index(drop=True)
last_user_logs_trend = user_log_trend2(last_user_logs_trend)

In [ ]:
last_user_logs_trend.head(3)

In [ ]:
last_user_logs_trend.to_csv('extra_features2.csv', index=False)

In [ ]:
# user_logs sum & mean fater 2-1
df_iter = pd.read_csv('user_logs.csv/user_logs.csv', low_memory=False, iterator=True, chunksize=10000000)
last_user_logs = []
i = 0 
for df in df_iter:
    if len(df)>0:
        print(df.shape)
        p = Pool(cpu_count())
        df = p.map(user_log_count, np.array_split(df, cpu_count()))   
        df = pd.concat(df, axis=0, ignore_index=True).reset_index(drop=True)
        df = user_log_count2(df)
        p.close(); p.join()
        last_user_logs.append(df)
        print('...', df.shape)
        df = []
        gc.collect()

last_user_logs = pd.concat(last_user_logs, axis=0, ignore_index=True).reset_index(drop=True)
last_user_logs = user_log_count2(last_user_logs)

In [ ]:
last_user_logs.head(3)

In [ ]:
last_user_logs.to_csv('extra_features.csv', index=False)

In [12]:
# user num sum after 3-1
df_iter = pd.read_csv('user_logs.csv/user_logs_v2.csv', low_memory=False, iterator=True, chunksize=10000000)
last_user_logs = []
i = 0 
for df in df_iter:
    if len(df)>0:
        print(df.shape)
        p = Pool(cpu_count())
        df = p.map(func.user_log_sum, np.array_split(df, cpu_count()))   
        df = pd.concat(df, axis=0, ignore_index=True).reset_index(drop=True)
        df = func.user_log_sum2(df)
        p.close(); p.join()
        last_user_logs.append(df)
        print('...', df.shape)
        df = []
        gc.collect()

last_user_logs = pd.concat(last_user_logs, axis=0, ignore_index=True).reset_index(drop=True)
last_user_logs = func.user_log_sum2(last_user_logs)

(10000000, 9)
... (661542, 8)
(10000000, 9)
... (585393, 8)
(10000000, 9)
... (649722, 8)
(10000000, 9)
... (662229, 8)
(10000000, 9)
... (613887, 8)
(10000000, 9)
... (580158, 8)
(10000000, 9)
... (634413, 8)
(10000000, 9)
... (662149, 8)
(10000000, 9)
... (563477, 8)
(10000000, 9)
... (595551, 8)
(10000000, 9)
... (662440, 8)
(10000000, 9)
... (661494, 8)
(10000000, 9)
... (546180, 8)
(10000000, 9)
... (596466, 8)
(10000000, 9)
... (663025, 8)
(10000000, 9)
... (631776, 8)
(10000000, 9)
... (596415, 8)
(10000000, 9)
... (640445, 8)
(10000000, 9)
... (662538, 8)
(10000000, 9)
... (600753, 8)
(10000000, 9)
... (593493, 8)
(10000000, 9)
... (661585, 8)
(10000000, 9)
... (628885, 8)
(10000000, 9)
... (578837, 8)
(10000000, 9)
... (626743, 8)
(10000000, 9)
... (662359, 8)
(10000000, 9)
... (624582, 8)
(10000000, 9)
... (579140, 8)
(10000000, 9)
... (648529, 8)
(10000000, 9)
... (661305, 8)
(10000000, 9)
... (607222, 8)
(10000000, 9)
... (633829, 8)
(10000000, 9)
... (661516, 8)
(10000000,

In [15]:
last_user_logs.count()

msno                 1431802
num_25_sum_1m        1431802
num_50_sum_1m        1431802
num_75_sum_1m        1431802
num_985_sum_1m       1431802
num_100_sum_1m       1431802
num_unq_sum_1m       1431802
total_secs_sum_1m    1431802
dtype: int64

In [16]:
last_user_logs.to_csv('extra_features4.csv', index=False)